# Assignment 4

In this assignment, you'll combine the assignment 3 data set with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list). The CSV file `fresh.csv` contains the fresh fruits and vegetables data you extracted in assignment 3.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it would be a good idea to use [caching][requests_cache].

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any Data.gov API. You may need the key again later in the quarter, so make sure you save it.

These modules may be useful:

* [requests](http://docs.python-requests.org/en/master/user/quickstart/)
* [requests_cache][]
* [urlparse](https://docs.python.org/2/library/urlparse.html)
* [pandas](http://pandas.pydata.org/pandas-docs/stable/)

[requests_cache]: https://pypi.python.org/pypi/requests-cache

Sites used: http://stackoverflow.com/questions/5188792/how-to-check-a-string-for-specific-characters
http://www.pythonforbeginners.com/dictionary/python-split
http://stackoverflow.com/questions/5438745/python-nan-and-inf-values

__Exercise 1.1.__ Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` should return this list:

```python
[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4},
 {u'ds': u'SR',
  u'group': u'Dairy and Egg Products',
  u'name': u'Egg, quail, whole, fresh, raw',
  u'ndbno': u'01140',
  u'offset': 5}]
```

As usual, make sure you document and test your function.

In [22]:
from urllib2 import Request, urlopen
from urlparse import urlparse, urlunparse
import requests, requests_cache
import pandas as pd
import json


# my API key
key = "ULxnv6kWU0vTif6L3wHrB5MIkQKj0PrM3IfgfWbG"

In [5]:
requests_cache.install_cache('food_cache')

In [35]:
# generate the query request to search for results

# Notes: The request will return a JSON object. Default sort = "r", aka relevance
# EXAMPLE URL: https://api.nal.usda.gov/ndb/search/?api_key=ULxnv6kWU0vTif6L3wHrB5MIkQKj0PrM3IfgfWbG&format=json&q=mushroom

maxResults = 5

def get_request(key, searchObject):
    urlbase = "https://api.nal.usda.gov/ndb/search/"
    params = {"q":searchObject}
    params.update({"format":"json"})
    params.update({"api_key":key})
    params.update({"max":maxResults})
    return requests.get(urlbase,params=params)
    #return urlbase + "/?" + "&".join("{}={}".format(a,b) for a,b in params.items())

get_request(key, "mushroom")

<Response [200]>

In [142]:
def ndb_search(searchObject):
    searchList = []
    request = get_request(key, searchObject)
    searchList = request.json()
    return searchList[u'list'][u'item']

ndb_search("mushroom")

[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'MUSHROOM GRAVY MADE WITH REAL MUSHROOMS, UPC: 011213168265',
  u'ndbno': u'45077161',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BARTOLINI, EXTRA VIRGIN OLIVE OIL WITH MUSHROOM OIL, PORCINI MUSHROOM OIL, UPC: 657739001541',
  u'ndbno': u'45168655',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'MUSHROOM HOUSE, DRIED OYSTER MUSHROOMS, UPC: 084348369166',
  u'ndbno': u'45184863',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'MUSHROOM HOUSE, SHIITAKE DRIED MUSHROOMS, UPC: 084348374160',
  u'ndbno': u'45184864',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'MUSHROOM HOUSE, DRIED MUSHROOMS, UPC: 084348396162',
  u'ndbno': u'45184865',
  u'offset': 4}]

In [40]:
# trying on quail eggs
ndb_search("quail eggs")

[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4}]

__Exercise 1.2.__ Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* You can convert the output of `ndb_search()` to a data frame with `pd.DataFrame()`.
* The string methods for [Python](https://docs.python.org/2/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

In [43]:
fresh = pd.read_csv("fresh.csv")
fresh.head()

,form,price_per_lb,yield,lb_per_cup,price_per_cup,food,type
0,Fresh1,0.333412,0.52,0.330693,0.212033,watermelon,fruit
1,Fresh1,0.535874,0.51,0.374786,0.393800,cantaloupe,fruit
2,Fresh1,1.377962,0.74,0.407855,0.759471,tangerines,fruit
3,Fresh1,2.358808,0.94,0.319670,0.802171,strawberries,fruit
4,Fresh1,1.827416,0.94,0.363763,0.707176,plums,fruit


After searching on the online page, I noticed the majority have the word are formatted "[fruit/vegetable], raw", so I will adjust my search to include that in the parameter. Entries with more than one word are separted by a ", ".

In order to find the fresh fruit or vegetable you are looking for, I am modifying the search term to make it appear on top. Modifications include:
<ul>
  <li>Adding '<b>, raw</b>' to the end of every entry</li>
  <li>Splitting multi-word entries with a comma</li>
  <li>Dealing with special cases</li>
</ul>

In [51]:
def modify_search_term(food):
    searchTerm = food
    if "_" in food: # two+ word i.e. "collard_greens"
        searchTerm = ", ".join(food.split('_'))
    return searchTerm + ", raw"

modify_search_term("lettuce_iceberg")

'lettuce, iceberg, raw'

In [53]:
# testing it out
ndb_search(modify_search_term("lettuce_iceberg"))

[{u'ds': u'SR',
  u'group': u'Vegetables and Vegetable Products',
  u'name': u'Lettuce, iceberg (includes crisphead types), raw',
  u'ndbno': u'11252',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'KROGER, FRESH SELECTIONS, COBB SALAD KIT FOR ONE, ROMAINE LETTUCE, HARD BOILED EGGS, TURKEY BREAST, ICEBERG LETTUCE, BACON, CHUNKY BLEU CHEESE DRESSING, UPC: 011110917379',
  u'ndbno': u'45112321',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'KROGER, FRESH SELECTIONS, AMERICAN BLEND; ROMAIN LETTUCE, ICEBERG LETTUCE, RED CABBAGE, CARROTS & RADISH, UPC: 011110910417',
  u'ndbno': u'45112330',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'TAYLOR FARMS, ICEBERG LETTUCE, UPC: 030223010098',
  u'ndbno': u'45181820',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'SHREDDED ICEBERG LETTUCE, UPC: 03003430195',
  u'nd

Write something to deal with when the group isn't <b>Vegetables and Vegetable Products</b> or <b>Fruits and Fruit Products</b>. Raise a flag to indicate that the raw fruit wasn't found.

In [65]:
# lettuce is declared below
lettuce

{u'ds': u'SR',
 u'group': u'Vegetables and Vegetable Products',
 u'name': u'Lettuce, iceberg (includes crisphead types), raw',
 u'ndbno': u'11252',
 u'offset': 0}

In [114]:
def check_if_found(foodJson):
    foodGroup = foodJson[u'group']
    if (foodGroup == u'Vegetables and Vegetable Products') or (foodGroup == u'Fruits and Fruit Juices'):
        return True
    else:
        return False
    
check_if_found(lettuce)

True

In [76]:
def grab_correct_entry(jsonList):
    for result in jsonList:
        if check_if_found(result):
            return result
    return None

In [121]:
# main function to get ndb number

def get_ndbno(entry):
    searchTerm = modify_search_term(entry)
    searchResults = ndb_search(searchTerm)  
    
    foodEntry = grab_correct_entry(searchResults)
    if foodEntry is None:
        return float('NaN')
    return foodEntry['ndbno']
    
            
    
    #except NameError:
    #    print "No results found. Check your spelling"

lettuce = get_ndbno("lettuce_iceberg")

Add Error handling when:
<ul>
    <li>you entered the wrong entry into the search bar.</li>
    <li>you didn't get one of the raw unbranded fruits/vegetables.</li>
</ul>

In [ ]:
### IGNORE THIS ###
# main function to get ndb number

def get_ndbno(entry):
    searchTerm = modify_search_term(entry)
    searchResults = ndb_search(searchTerm)
    #except NameError:
    #    print "No results found. Check your spelling"
    return searchResults

lettuceList = ndb_search(modify_search_term("lettuce_iceberg"))
lettuce = lettuceList[0] # know this is the right entry

In [122]:
get_ndbno('watermelon')

u'09326'

In [124]:
ndbNumbers = {}
for food in fresh['food']:
    try:
        ndbNumbers[food] = get_ndbno(food)
    except:
        pass

In [138]:
ndbNumbers # didn't get duplicates: cabbage, cucumbers; and kiwi

{'acorn_squash': u'11482',
 'apples': u'09003',
 'apricots': u'09021',
 'artichoke': u'11226',
 'asparagus': u'11011',
 'avocados': u'09038',
 'bananas': u'09040',
 'blackberries': u'09042',
 'blueberries': u'09050',
 'brussels_sprouts': u'11098',
 'butternut_squash': u'11485',
 'cabbage': u'11503',
 'cantaloupe': u'09181',
 'cherries': u'09276',
 'collard_greens': u'11161',
 'corn_sweet': u'11900',
 'cucumbers': u'11206',
 'grapefruit': u'09117',
 'grapes': u'09129',
 'green_beans': u'11052',
 'green_peppers': u'11333',
 'honeydew': u'09184',
 'kale': u'11233',
 'lettuce_iceberg': u'11252',
 'mangoes': u'09176',
 'mustard_greens': u'11270',
 'nectarines': u'09191',
 'okra': u'11278',
 'onions': u'11282',
 'oranges': u'09201',
 'papaya': u'09226',
 'peaches': u'09236',
 'pears': u'09252',
 'pineapple': u'09266',
 'plums': u'09279',
 'pomegranate': u'09286',
 'potatoes': u'11362',
 'radish': u'11429',
 'raspberries': u'09302',
 'red_peppers': u'11821',
 'strawberries': u'09316',
 'summe

__Exercise 1.3.__ Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{u'group': u'Proximates',
 u'measures': [{u'eqv': 165.0,
   u'eunit': u'g',
   u'label': u'cup, sliced',
   u'qty': 1.0,
   u'value': u'143.93'},
  {u'eqv': 66.0,
   u'eunit': u'g',
   u'label': u'fruit (2-1/8" dia)',
   u'qty': 1.0,
   u'value': u'57.57'},
  {u'eqv': 151.0,
   u'eunit': u'g',
   u'label': u'NLEA serving',
   u'qty': 1.0,
   u'value': u'131.72'}],
 u'name': u'Water',
 u'nutrient_id': u'255',
 u'unit': u'g',
 u'value': u'87.23'}
```

Be sure to document and test your function.

In [181]:
def ndb_report(ndbNumber):
    url = "https://api.nal.usda.gov/ndb/V2/reports/"
    querystring = {"ndbno":"09236","api_key":"ULxnv6kWU0vTif6L3wHrB5MIkQKj0PrM3IfgfWbG","format":"json","type":"b"}
    response = requests.get(url, params=querystring)
    return response.json()[u'foods'][0]['food']['nutrients']


test = ndb_report(ndbNumbers.values()[0])
test

[{u'group': u'Proximates',
  u'measures': [{u'eqv': 154.0,
    u'eunit': u'g',
    u'label': u'cup slices',
    u'qty': 1.0,
    u'value': u'136.86'},
   {u'eqv': 130.0,
    u'eunit': u'g',
    u'label': u'small (2-1/2" dia)',
    u'qty': 1.0,
    u'value': u'115.53'},
   {u'eqv': 150.0,
    u'eunit': u'g',
    u'label': u'medium (2-2/3" dia)',
    u'qty': 1.0,
    u'value': u'133.31'},
   {u'eqv': 175.0,
    u'eunit': u'g',
    u'label': u'large (2-3/4" dia)',
    u'qty': 1.0,
    u'value': u'155.52'},
   {u'eqv': 224.0,
    u'eunit': u'g',
    u'label': u'extra large (3" dia)',
    u'qty': 1.0,
    u'value': u'199.07'},
   {u'eqv': 147.0,
    u'eunit': u'g',
    u'label': u'NLEA serving',
    u'qty': 1.0,
    u'value': u'130.64'}],
  u'name': u'Water',
  u'nutrient_id': u'255',
  u'unit': u'g',
  u'value': u'88.87'},
 {u'group': u'Proximates',
  u'measures': [{u'eqv': 154.0,
    u'eunit': u'g',
    u'label': u'cup slices',
    u'qty': 1.0,
    u'value': u'60'},
   {u'eqv': 130.0,
   

__Exercise 1.4.__ Which foods provide the best combination of price, yield, and nutrition? You can use kilocalories as a measure of "nutrition" here, but more a detailed analysis is better. Use plots to support your analysis.